In [23]:
import pandas as pd
import datetime as dt

pd.options.display.max_columns=1000
pd.options.display.max_rows = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%run configs.ipynb

In [24]:
# parameter cell
file_name = 'oct_21.csv'

# RRC Prod

## Ingest

In [25]:
production = pd.read_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', header = 5)

In [26]:
production.head()

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
0,NaN,Oil,8.0,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,140,26,NaN,NaN,166,0,NaN,NaN
1,NaN,Oil,8.0,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,18,0,NaN,NaN,18,0,NaN,NaN
2,NaN,Oil,8.0,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,261,87,183,1.0,165,"1,699","1,699",2.0
3,Y,Oil,8.0,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"1,721","7,857","8,126",0.0,"1,452","19,723","19,723",2.0
4,NaN,Oil,8.0,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,507,870,"1,184",0.0,193,"3,832","3,832",2.0


## Transform RRC

### Filter to District

In [27]:
#converting data field to string to handle varying schema interpreations 
#(certain months field will be interpreted as float and string depending sample rows pandas uses

production = production.loc[production['District'].astype(str).isin(['8.0', '08', '8'])]

In [28]:
production

,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,Volume,Code,On Hand End of Month,Formation Production,Volume.1,Code.1
0,NaN,Oil,8.0,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,140,26,NaN,NaN,166,0,NaN,NaN
1,NaN,Oil,8.0,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,18,0,NaN,NaN,18,0,NaN,NaN
2,NaN,Oil,8.0,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,261,87,183,1.0,165,"1,699","1,699",2.0
3,Y,Oil,8.0,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"1,721","7,857","8,126",0.0,"1,452","19,723","19,723",2.0
4,NaN,Oil,8.0,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,507,870,"1,184",0.0,193,"3,832","3,832",2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,NaN,Gas,8.0,147187.0,WADDELL (GRAYBURG),"WADDELL, W.N.",859,NaN,8988.0,8,31,37,1.0,2,736,736,2.0
503,NaN,Gas,8.0,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,NaN,NaN,0,0,NaN,NaN
504,NaN,Gas,8.0,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,8,31,37,1.0,2,606,606,2.0
506,NaN,Gas,8.0,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,8,31,37,1.0,2,784,784,2.0


In [29]:
production.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452 entries, 0 to 508
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Multiple Reports            113 non-null    object 
 1   Lease Type                  407 non-null    object 
 2   District                    452 non-null    float64
 3   RRC Identifier              452 non-null    float64
 4   Field Name                  452 non-null    object 
 5   Lease Name                  452 non-null    object 
 6   Gas Well ID                 232 non-null    object 
 7   Lease Total                 12 non-null     object 
 8   Commingle Permit No.        263 non-null    float64
 9   On Hand Beginning of Month  400 non-null    object 
 10  Production                  452 non-null    object 
 11  Volume                      180 non-null    object 
 12  Code                        180 non-null    float64
 13  On Hand End of Month        452 non

### Clean Oil Production Columns and Convert to Int

In [30]:
production['Volume'] = production.loc[:,'Volume'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))
production['Volume.1'] = production.loc[:,'Volume.1'].fillna(0).apply(lambda x: int(str(x).replace(',', '')))

### Rename Volume to rrc_oil_volume

In [31]:
production.rename(columns = {'Volume': 'rrc_oil_volume'}, inplace = True)
production.rename(columns = {'Volume.1': 'rrc_gas_volume'}, inplace = True)

In [32]:
production['rrc_oil_volume'].sum()
production['rrc_gas_volume'].sum()

361442

1252917

### Normalize Field and Reservoir Names with Text Cleaning

In [33]:
production["Normalized_Field_Name"] = production['Field Name'].dropna()\
                                                            .apply(lambda x: str(x).split('(')[0]\
                                                                                    .split(',')[0]\
                                                                                    .strip())

In [34]:
production["Normalized_Reservoir_Name"] = production['Field Name'].dropna()\
                                        .apply(lambda x: str(x).split('(')[-1]\
                                                               .strip().replace(')',''))

### Get Date from Production File

In [35]:
def get_date_from_csv(filepath, col_number, row_number):
    str_date = pd.read_csv(filepath, usecols = [col_number]).iloc[row_number].values[0]
    return str_date.strip()

In [36]:
get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1)

'Oct 2021'

### Add Production Date to Production Data Frame

In [37]:
production.insert(0, 'Date', get_date_from_csv(f'{raw_data_folder}/rrc_prod_blackbeard/2021/{file_name}', 3, 1))

In [38]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name
0,Oct 2021,NaN,Oil,8.0,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,140,26,0,NaN,166,0,0,NaN,ARMER,TUBB
1,Oct 2021,NaN,Oil,8.0,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,18,0,0,NaN,18,0,0,NaN,ARMER,6350
2,Oct 2021,NaN,Oil,8.0,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,261,87,183,1.0,165,"1,699",1699,2.0,ARMER,6350
3,Oct 2021,Y,Oil,8.0,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"1,721","7,857",8126,0.0,"1,452","19,723",19723,2.0,ARMER,6350
4,Oct 2021,NaN,Oil,8.0,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,507,870,1184,0.0,193,"3,832",3832,2.0,ARMER,6350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,Oct 2021,NaN,Gas,8.0,147187.0,WADDELL (GRAYBURG),"WADDELL, W.N.",859,NaN,8988.0,8,31,37,1.0,2,736,736,2.0,WADDELL,GRAYBURG
503,Oct 2021,NaN,Gas,8.0,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG
504,Oct 2021,NaN,Gas,8.0,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,8,31,37,1.0,2,606,606,2.0,WADDELL,GRAYBURG
506,Oct 2021,NaN,Gas,8.0,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,8,31,37,1.0,2,784,784,2.0,WADDELL,GRAYBURG


### Add columns for net production to Trust

In [39]:
production['Net Oil Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50
production['Net Oil Sales Volume to Trust (RRC)'] = production['rrc_oil_volume'] * 0.50 * 0.75

In [40]:
production['Net Gas Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50
production['Net Gas Sales Volume to Trust (RRC)'] = production['rrc_gas_volume'] * 0.50 * 0.75

In [41]:
production

,Date,Multiple Reports,Lease Type,District,RRC Identifier,Field Name,Lease Name,Gas Well ID,Lease Total,Commingle Permit No.,On Hand Beginning of Month,Production,rrc_oil_volume,Code,On Hand End of Month,Formation Production,rrc_gas_volume,Code.1,Normalized_Field_Name,Normalized_Reservoir_Name,Net Oil Volume to Trust (RRC),Net Oil Sales Volume to Trust (RRC),Net Gas Volume to Trust (RRC),Net Gas Sales Volume to Trust (RRC)
0,Oct 2021,NaN,Oil,8.0,32907.0,ARMER (TUBB),"MCCAMEY, G. B. ""A"" (NCT-B)",NaN,NaN,7024.0,140,26,0,NaN,166,0,0,NaN,ARMER,TUBB,0.0,0.000,0.0,0.000
1,Oct 2021,NaN,Oil,8.0,20550.0,ARMER (6350),"MCCAMEY, G. B., -A- /NCT-A/",NaN,NaN,7024.0,18,0,0,NaN,18,0,0,NaN,ARMER,6350,0.0,0.000,0.0,0.000
2,Oct 2021,NaN,Oil,8.0,45314.0,ARMER (6350),RAYDEN MCCAMEY,NaN,NaN,NaN,261,87,183,1.0,165,"1,699",1699,2.0,ARMER,6350,91.5,68.625,849.5,637.125
3,Oct 2021,Y,Oil,8.0,48393.0,ARMER (6350),LANDLUBBER,NaN,NaN,8026.0,"1,721","7,857",8126,0.0,"1,452","19,723",19723,2.0,ARMER,6350,4063.0,3047.250,9861.5,7396.125
4,Oct 2021,NaN,Oil,8.0,49425.0,ARMER (6350),6 POUNDER NE,NaN,NaN,8026.0,507,870,1184,0.0,193,"3,832",3832,2.0,ARMER,6350,592.0,444.000,1916.0,1437.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,Oct 2021,NaN,Gas,8.0,147187.0,WADDELL (GRAYBURG),"WADDELL, W.N.",859,NaN,8988.0,8,31,37,1.0,2,736,736,2.0,WADDELL,GRAYBURG,18.5,13.875,368.0,276.000
503,Oct 2021,NaN,Gas,8.0,190864.0,WADDELL (GRAYBURG),"WADDELL, W.N.",890,NaN,8988.0,0,0,0,NaN,0,0,0,NaN,WADDELL,GRAYBURG,0.0,0.000,0.0,0.000
504,Oct 2021,NaN,Gas,8.0,190926.0,WADDELL (GRAYBURG),"WADDELL, W.N.",293,NaN,3501.0,8,31,37,1.0,2,606,606,2.0,WADDELL,GRAYBURG,18.5,13.875,303.0,227.250
506,Oct 2021,NaN,Gas,8.0,205775.0,WADDELL (GRAYBURG),"WADDELL, W. N.",1200,NaN,8092.0,8,31,37,1.0,2,784,784,2.0,WADDELL,GRAYBURG,18.5,13.875,392.0,294.000


### Write to File

In [42]:
production.to_csv(f'{processed_data_folder}/rrc_prod/{file_name}', index = False)

## Aggregated EDA

In [43]:
production.groupby(['Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'})\
                                            .sort_values(by = 'rrc_oil_volume', ascending= False)\
                                            .head(15)

,rrc_oil_volume
Normalized_Field_Name,
SAND HILLS,213134
MONAHANS,73275
ARMER,31688
DUNE,17158
MARSTON RANCH,9117
RUNNING W,5582
WADDELL,4810
UNIVERSITY WADDELL,2425
CORDONA LAKE,2124


In [44]:
production.groupby(['Normalized_Reservoir_Name', 'Normalized_Field_Name']).agg({'rrc_oil_volume': 'sum'}).sort_values(by = 'rrc_oil_volume', ascending= False).sort_index()

rrc_oil_volume
Normalized_Reservoir_Name Normalized_Field_Name                
6350                      ARMER                           31688
7900                      EDWARDS -04-                        0
CLEAR FORK                LEA                               331
                          MONAHANS                            0
                          SAND HILLS                      39232
CLEAR FORK 4070           SAND HILLS                        642
CLEAR FORK, LOWER         MCKEE                               0
CLEAR FORK, MID.          MCKEE                               0
CLEARFORK                 MARSTON RANCH                    9117
                          MONAHANS                        67562
                          RM                                  0
COLBY                     KEYSTONE                            0
CONSOLIDATED              KEYSTONE                            0
CORDONA LAKE              CORDONA LAKE                       92
COWDEN, NORTH             COWDEN                              0
DEV.                      CORDONA LAKE                     1832
DEVONIAN                  CORDONA LAKE                      143
                          UNIVERSITY WADDELL               1348
                          WADDELL                           202
DUNE                      DUNE                            17144
ELLENBURGER               KERMIT                              0
                          SAND HILLS                        954
                          UNIVERSITY WADDELL                  0
GLORIETA                  MONAHANS                          663
GRAYBURG                  GOLDSMITH                           0
                          WADDELL                           354
HOLT                      RUNNING W                          25
                          SAND HILLS                          0
JUDKINS                   SAND HILLS                        303
KERMIT                    KERMIT                              0
MCKEE                     BLOCK 27                            0
                          KERMIT                              0
                          RUNNING W                           0
MCKNIGHT                  SAND HILLS                      47479
MONAHANS, NORTH           MONAHANS                         5050
ORDOVICIAN                SAND HILLS                          0
PENN UPPER                BLOCK 31                          530
PENN.                     G. P. M.                            0
                          UNIVERSITY WADDELL                 56
QUEEN                     GOLDSMITH                           0
SAN ANDRES                C-BAR                               0
                          DELL                              170
                          KEYSTONE                            0
                          LEA                                84
                          SHAFTER LAKE                        0
SAN ANDRES CONS.          CRAWAR                              0
SAN ANDRES, CON.          GOLDSMITH                           0
SAN ANGELO                LEA                              1014
                          SAND HILLS                       1952
SAN ANGELO, UPPER         SAND HILLS                      38843
TUBB                      -Y-                                 0
                          ARMER                               0
                          BLOCK B-21                          0
                          RUNNING W                        1735
                          SAND HILLS                      42940
TUBB 4500                 CORDONA LAKE                       57
WADDELL                   RUNNING W                        3184
                          WADDELL                          4254
WADDELL SD.               UNIVERSITY WADDELL               1021
WICHITA-ALBANY            RUNNING W                         638
                          SAND HILLS                      23051
WICHITA-ALBANY, G